## Part V: Training a Model with Data Programming

In [1]:
# If necessary:
import os
os.remove('snorkel.db');
os.system('cp snorkel.db\ features snorkel.db');

In [2]:
%load_ext autoreload
%autoreload 2

from snorkel import SnorkelSession
session = SnorkelSession()

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [3]:
from snorkel.models import candidate_subclass

Part_Temp = candidate_subclass('Part_Temp', ['part','temp'])

### Loading the `CandidateSet`, feature matrix, and label matrix

In [4]:
from snorkel.models import CandidateSet
train = session.query(CandidateSet).filter(
    CandidateSet.name == 'Hardware Training Candidates').one()

In [5]:
from snorkel.annotations import FeatureManager

feature_manager = FeatureManager()

%time F_train = feature_manager.load(session, train, 'Train Features')

CPU times: user 54.7 s, sys: 2.01 s, total: 56.7 s
Wall time: 58.8 s


In [6]:
from snorkel.annotations import LabelManager

label_manager = LabelManager()

%time L_train = label_manager.load(session, train, 'LF Labels 2')

CPU times: user 1.12 s, sys: 73.3 ms, total: 1.2 s
Wall time: 1.3 s


## Train Generative Model

In [7]:
from snorkel.learning import NaiveBayes

gen_model = NaiveBayes()
# gen_model.train(L_train, n_iter=3000, rate=1e-5)
gen_model.train(L_train, n_iter=15000, rate=1e-2)

Training marginals (!= 0.5):	26544
Features:			9
Begin training for rate=0.01, mu=1e-06
	Learning epoch = 0	Gradient mag. = 0.384222
	Learning epoch = 250	Gradient mag. = 0.550661
	Learning epoch = 500	Gradient mag. = 0.756780
	Learning epoch = 750	Gradient mag. = 0.970949
	Learning epoch = 1000	Gradient mag. = 0.913820
	Learning epoch = 1250	Gradient mag. = 0.751362
	Learning epoch = 1500	Gradient mag. = 0.657093
	Learning epoch = 1750	Gradient mag. = 0.596412
	Learning epoch = 2000	Gradient mag. = 0.466138
	Learning epoch = 2250	Gradient mag. = 0.330497
	Learning epoch = 2500	Gradient mag. = 0.275490
	Learning epoch = 2750	Gradient mag. = 0.243697
	Learning epoch = 3000	Gradient mag. = 0.220060
	Learning epoch = 3250	Gradient mag. = 0.200634
	Learning epoch = 3500	Gradient mag. = 0.184074
	Learning epoch = 3750	Gradient mag. = 0.169722
	Learning epoch = 4000	Gradient mag. = 0.157158
	Learning epoch = 4250	Gradient mag. = 0.146076
	Learning epoch = 4500	Gradient mag. = 0.136241
	Learn

In [8]:
gen_model.save(session, 'Generative Params')

In [9]:
train_marginals = gen_model.marginals(L_train)

## Train Discriminative Model

In [10]:
from snorkel.learning import LogReg

disc_model = LogReg()
disc_model.train(F_train, train_marginals, n_iter=5000, rate=1e-3)

Training marginals (!= 0.5):	26544
Features:			14864
Using gradient descent...
	Learning epoch = 0	Step size = 0.001
	Loss = 18398.898761	Gradient magnitude = 36095.095696
	Learning epoch = 100	Step size = 0.000904792147114
	Loss = 29.745870	Gradient magnitude = 3.156072
	Learning epoch = 200	Step size = 0.000818648829479
	Loss = 29.289281	Gradient magnitude = 1.875095
	Learning epoch = 300	Step size = 0.000740707032156
	Loss = 29.044757	Gradient magnitude = 1.703231
	Learning epoch = 400	Step size = 0.000670185906007
	Loss = 28.847388	Gradient magnitude = 1.651350
	Learning epoch = 500	Step size = 0.000606378944861
	Loss = 28.676077	Gradient magnitude = 1.629568
	Learning epoch = 600	Step size = 0.000548646907485
	Loss = 28.524033	Gradient magnitude = 1.618579
	Learning epoch = 700	Step size = 0.000496411413431
	Loss = 28.387882	Gradient magnitude = 1.612317
	Learning epoch = 800	Step size = 0.00044914914861
	Loss = 28.265453	Gradient magnitude = 1.608415
	Learning epoch = 900	Step si

In [11]:
disc_model.w.shape

(14864,)

In [12]:
%time disc_model.save(session, "Discriminative Params")

CPU times: user 2.6 s, sys: 67.5 ms, total: 2.67 s
Wall time: 2.7 s


## Assess Performance on Development Set

In [13]:
from snorkel.models import CandidateSet
dev = session.query(CandidateSet).filter(
    CandidateSet.name == 'Hardware Development Candidates').one()

### Loading the Development Feature Matrix

In [14]:
from snorkel.annotations import FeatureManager

feature_manager = FeatureManager()
%time F_dev = feature_manager.load(session, dev, 'Train Features')

CPU times: user 43.2 s, sys: 2.23 s, total: 45.4 s
Wall time: 47.3 s


In [15]:
L_dev = label_manager.load(session, dev, "Hardware Development Labels -- Gold")

In [16]:
gold_dev_set = session.query(CandidateSet).filter(
    CandidateSet.name == 'Hardware Development Candidates -- Gold').one()

In [17]:
F_dev.shape

(17804, 14864)

In [18]:
tp, fp, tn, fn = disc_model.score(F_dev, L_dev, gold_dev_set)

Calibration plot:
Recall-corrected Noise-aware Model
Pos. class accuracy: 0.979873717443
Neg. class accuracy: 0.918969849246
Corpus Precision 0.828
Corpus Recall    0.98
Corpus F1        0.898
----------------------------------------
TP: 4966 | FP: 1032 | TN: 11704 | FN: 102

Recall-corrected Noise-aware Model
Pos. class accuracy: 0.979873717443
Neg. class accuracy: 0.918969849246
Corpus Precision 0.828
Corpus Recall    0.98
Corpus F1        0.898
----------------------------------------
TP: 4966 | FP: 1032 | TN: 11704 | FN: 102



/Users/bradenhancock/anaconda/lib/python2.7/site-packages/numpy/core/_methods.py:59: RuntimeWarning: Mean of empty slice.
  warnings.warn("Mean of empty slice.", RuntimeWarning)


In [19]:
from hardware_utils import part_error_analysis, table_info
#     print c[0]
#     print c[0].parent
#     print (c[0].parent.row_num, c[0].parent.col_num)
#     print c[1]
#     print c[1].parent
#     print (c[1].parent.row_num, c[1].parent.col_num)

In [21]:
# for c in list(fp)[:5]:
#     print c
# c = list(fp)[10]
# error_analysis(c)
# c = list(fp)[10]
# part_error_analysis(c)
# print c.temp.parent.row_num

# for c in list(fp)[:5]:
#     print c
# print c.temp.parent.html_anc_tags

In [22]:
from snorkel.models import Corpus
from hardware_utils import entity_level_f1
import os

gold_file = os.environ['SNORKELHOME'] + '/tutorials/tables/data/hardware/hardware_gold.csv'
corpus = session.query(Corpus).filter(Corpus.name == 'Hardware Development').one()
(TP, FP, FN) = entity_level_f1(tp, fp, tn, fn, gold_file, corpus, 'stg_temp_min')

Scoring on Entity-Level Gold Data
Corpus Precision 0.808
Corpus Recall    1.0
Corpus F1        0.894
----------------------------------------
TP: 135 | FP: 32 | FN: 0



In [23]:
print FP

set([(u'INFINEON-BC817KSERIES_BC818KSERIES-DS-V01_01-EN', u'BC817K-40W', u'-70'), (u'INFINEON-BC817KSERIES_BC818KSERIES-DS-V01_01-EN', u'BC817K-40', u'-70'), (u'INFINEON-BC857SERIES_BC858SERIES_BC859SERIES_BC860SERIES-DS-V01_01-EN', u'BC857CW', u'-60'), (u'INFINEON-BC857SERIES_BC858SERIES_BC859SERIES_BC860SERIES-DS-V01_01-EN', u'BC858C', u'-60'), (u'INFINEON-BC817KSERIES_BC818KSERIES-DS-V01_01-EN', u'BC817K-16W', u'-70'), (u'INFINEON-BC817KSERIES_BC818KSERIES-DS-V01_01-EN', u'BC817K-25W', u'-70'), (u'INFINEON-BC857SERIES_BC858SERIES_BC859SERIES_BC860SERIES-DS-V01_01-EN', u'BC857B', u'-60'), (u'INFINEON-BC857SERIES_BC858SERIES_BC859SERIES_BC860SERIES-DS-V01_01-EN', u'BC858B', u'-60'), (u'INFINEON-BC857SERIES_BC858SERIES_BC859SERIES_BC860SERIES-DS-V01_01-EN', u'BC857BL3', u'-60'), (u'INFINEON-BC817KSERIES_BC818KSERIES-DS-V01_01-EN', u'BC817K-25', u'-70'), (u'INFINEON-BC857SERIES_BC858SERIES_BC859SERIES_BC860SERIES-DS-V01_01-EN', u'BC858CW', u'-60'), (u'INFINEON-BC857SERIES_BC858SERIES_BC

In [ ]:
import os
os.system('cp snorkel.db snorkel.db\ final');

The End.